## Mean Average Percentange Error (MAPE) calculations

This notebook is used to interpolate between six stationary sensor values, perform k-fold cross validation on CNN predictions, and map statistical and CNN prediction results. Mean Average Percentage Error (MAPE) is calculated for interpolation and CNN predictions.

In [2]:
import numpy as np
from constants import Constants
from grid_definition import GridDefinition
import math
import csv
import pandas as pd
from scipy import interpolate

import matplotlib
import matplotlib.pyplot as plt
import os
from osm_reader import OSMReader
import folium
from data_download import DataDownloader
import random

import pykrige.kriging_tools as kt
from pykrige.uk import UniversalKriging
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import Rbf

In [3]:
constants = Constants()
gridDefinition = GridDefinition()
gridDefinition.init()
osm_reader = OSMReader()
osm_reader.init()

corners = constants.getCorners()
staticCoords = constants.getStaticCoords()
GRID_SIZE = constants.getGridSize()

topLat = gridDefinition.getTopLat()
leftLon = gridDefinition.getLeftLon()

bottomLat = gridDefinition.getBottomLat()
rightLon = gridDefinition.getRightLon()

height = topLat - bottomLat
width = rightLon - leftLon

heightInterval = height / GRID_SIZE
widthInterval = width / GRID_SIZE

np.set_printoptions(precision=2, suppress=True)

data_downloader = DataDownloader()

PMstrs = ['PM1', 'PM2.5', 'PM10']

# Calculate MAPE for Interpolation

In [8]:
errors = []
MAPPING_ON = False
DOUBLE_COLLECTION = False
interp_strs = ["linear", "smooth", "UK", "OK", "RBF"]

data_collected = constants.getNineteenDates() 

if (MAPPING_ON):
    cells = osm_reader.getGeoCells()

for toDelete in range(1):
    pm = 1
    for i in [0, 1, 2, 3, 4]:
        INTERP_TYPE = i
        errors = []
        for start_date in data_collected:
            interpolate_PM(start_date, pm, 0)
        
        if (errors):
            print(str(interp_strs[i]) + "," +  str(np.average(errors)))
            
        print(errors)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ryanegan/Documents/diss/projectZoe/data/label/train/20180703/20180703_grid20.csv'

# Calculate MAPE for CNN Predictions

In [5]:
data_collected = constants.getNineteenDates() 

kernel_sizes = [1, 2, 3]
learning_rate = 0.005 

MAPPING_ON = False

if (MAPPING_ON):
    cells = osm_reader.getGeoCells()
    
i = 1

for kernel_size in kernel_sizes:
    print(PMstrs[i] + ", kernel size: " + str(kernel_size) +", learning_rate: " + str(learning_rate))
    errors = []
    r2s = []
    
    for date in data_collected:
        ML_dir = '../results/19tune/learning_rate/learning_rate_' +str(learning_rate) + '/kernel_size_' +str(kernel_size) + '/valid_date_'+str(date)+'/'
        ML_pred_file = ML_dir + "PM2.5_prediction.csv"

        pd_df=pd.read_csv(ML_pred_file, sep=' ',header=None)
        PM_pred = pd_df.values
        label_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/label/train/"+str(date)+"/"
        sids = ['XXM007', 'XXM008']
    
        label_file = label_dir + sids[1] + "_" + str(date) + "_" + PMstrs[i] + "_grid" + str(GRID_SIZE) + ".csv"
        labels_found = False
        
        if (os.path.isfile(label_file)):
            sid_plot = sids[1]
            labels_found = True
        else:
            label_file = label_dir + sids[0] + "_" + str(date) + "_" + PMstrs[i] + "_grid" + str(GRID_SIZE) + ".csv"
            if (os.path.isfile(label_file)):
                sid_plot = sids[0]
                labels_found = True
        if(labels_found):
            pd_df=pd.read_csv(label_file, sep=',',header=None, skiprows=3)
            labels = pd_df.values
        else:
            print("no labels found")

        pd_df=pd.read_csv(label_file, sep=',',header=None, skiprows=3)
        PM_true = pd_df.values
        error = calculateError(PM_true, PM_pred)
        errors.append(error)
        
        if (MAPPING_ON):
            map_dir = "/Users/ryanegan/Documents/diss/projectZoe/images/maps/interpolations/CNN/"+str(date)+"/"
            maxPM = np.max(PM_pred)
            PM_index = 1
            pred_map = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
            labelAndPlotPersonalData(map_dir, PM_pred, sid_plot, PM_index, date)
        
        r2 = calculateR2(PM_true, PM_pred)
        r2s.append(r2)
    
    overallError = np.average(errors)    
    print(" kernel size : , " + str(kernel_size) + " , " + str(overallError))
    
    ML_train_cost = ML_dir + "PM2.5_train_cost.csv"
    ML_val_cost = ML_dir + "PM2.5_valid_cost.csv"
    train_df=pd.read_csv(ML_train_cost, sep=' ',header=None)
    valid_df=pd.read_csv(ML_val_cost, sep=' ',header=None)
    plt.figure() 
    valid_df.plot()
    ax_val = valid_df.plot()

PM2.5, kernel size: 1, learning_rate: 0.005
no labels found


FileNotFoundError: [Errno 2] File b'/Users/ryanegan/Documents/diss/projectZoe/data/label/train/20180629/XXM007_20180629_PM2.5_grid50.csv' does not exist: b'/Users/ryanegan/Documents/diss/projectZoe/data/label/train/20180629/XXM007_20180629_PM2.5_grid50.csv'

## Functions for interpolation and machine learning MAPE calculations

In [6]:
#Functions for interpolation and machine learning MAPE calculations
def interpolate_PM(start_date, PM_index, toDelete):
    PM = []
    labels_to_plot = []
    x, y, z = readStaticData(start_date, PM_index)
    
    label_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/label/"+str(start_date)+"/"
    sids = ['XXM007', 'XXM008']
    
    labels = []

    if (DOUBLE_COLLECTION):
        index_interp = random.choice((0,1))
        index_plot = not index_interp
        
        sid_interp = sids[index_interp]
        sid_plot = sids[index_plot]

        plot_filename = label_dir + sid_plot + "_" + str(start_date) + "_" + PMstrs[PM_index] + "_grid" + str(GRID_SIZE) + ".csv"
        if (os.path.isfile(plot_filename)):
            labels_to_plot.append(sid_plot)
            
        interp_filename = label_dir + sid_interp + "_" + str(start_date) + "_" + PMstrs[PM_index] + "_grid" + str(GRID_SIZE) + ".csv"
        if (os.path.isfile(interp_filename)):
            x, y, z = readPersonalData(x, y, z, start_date, sid_interp, PM_index)
    
    interp_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/data/train/"+str(start_date)+"/"+ PMstrs[PM_index] + "/"
    if not os.path.exists(interp_dir):
        os.makedirs(interp_dir)
            
    xnew = np.arange(0, GRID_SIZE, 1.0)
    ynew = np.arange(0, GRID_SIZE, 1.0)
    
    if (INTERP_TYPE == 0):
        f = interpolate.interp2d(x, y, z, kind='linear')
        filename = interp_dir  + str(start_date) + "_grid" + str(GRID_SIZE) + "_interp2D.csv"
        map_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/images/maps/interpolations/interp2d/"+str(start_date)+"/"
        PM_pred = f(xnew, ynew)

    elif (INTERP_TYPE == 1):
        f = interpolate.SmoothBivariateSpline(x, y, z, kx=1, ky=1)
        filename = interp_dir + str(start_date) + "_grid" + str(GRID_SIZE) + "_smoothspline.csv"
        map_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/images/maps/interpolations/SmoothBivariate/"+str(start_date)+"/"
        PM_pred = f(xnew, ynew)
    
    elif (INTERP_TYPE == 2):
        filename = interp_dir + str(start_date) + "_grid" + str(GRID_SIZE) + "_universalKriging.csv"
        map_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/images/maps/interpolations/UniversalKriging/"+str(start_date)+"/"
        UK = UniversalKriging(x, y, z, variogram_model='linear',drift_terms=['regional_linear'])
        znew, ss = UK.execute('grid', xnew, ynew)
        PM_pred = znew
    
    elif (INTERP_TYPE == 3):
        filename = interp_dir + str(start_date) + "_grid" + str(GRID_SIZE) + "_ordinaryKriging.csv"
        map_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/images/maps/interpolations/OrdinaryKriging/"+str(start_date)+"/"
        OK = OrdinaryKriging(x, y, z, variogram_model='linear', verbose=False, enable_plotting=False)
        znew, ss = OK.execute('grid', xnew, ynew)
        PM_pred = znew
        
    elif (INTERP_TYPE == 4):
        rbfi = Rbf(x, y, z, function='gaussian')
        PM_pred = rbfi(xnew, ynew)
        filename = interp_dir + str(start_date) + "_grid" + str(GRID_SIZE) + "_rbf.csv"
        map_dir = "/Users/zoepetard/Google Drive/Edinburgh/MscProj/FillingTheGaps/images/maps/interpolations/RBF/"+str(start_date)+"/"

        PM_pred = np.zeros((xnew.shape[0], ynew.shape[0]))
        for i in range(xnew.shape[0]):
            for j in range(ynew.shape[0]):
                PM_pred[j,i] = rbfi(xnew[i], ynew[j])

    if not os.path.exists(map_dir):
        os.makedirs(map_dir)

    PM_pred[PM_pred < 0.0] = 0.0 #Shouldn't have negative predictions
    maxPM = np.max(PM_pred)

    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile, lineterminator='\n')
        writer.writerows(PM_pred)

    if not DOUBLE_COLLECTION:
        filename = label_dir + sids[0] + "_" + str(start_date) + "_" + PMstrs[1] + "_grid" + str(GRID_SIZE) + ".csv"
        if (os.path.isfile(filename)):
            sid_plot = sids[0]
        else:
            sid_plot = sids[1]
    labelAndPlotPersonalData(map_dir, PM_pred, sid_plot, PM_index, start_date)
    
def readStaticData(start_date, PM_index):
    PM = []

    train_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/label/train/"+str(start_date)+"/"

    filename = train_dir + str(start_date) + "_grid" + str(GRID_SIZE) + ".csv"
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        medianTime = pd.to_datetime(next(reader)[0])
        averageTemp = next(reader)[0]
        averageHum = next(reader)[0]
    
        PM.append(next(reader))
        PM.append(next(reader))
        PM.append(next(reader))
    
    x = np.zeros(6)
    y = np.zeros(6)
    z = np.zeros(6)
    
    for num, coord in enumerate(staticCoords):
        col = math.floor((topLat - coord[0]) / heightInterval)
        row = math.floor((coord[1] - leftLon) / widthInterval)
        x[num] = row
        y[num] = col
        z[num] = PM[PM_index][num] 
    
    return x, y, z

def readPersonalData(x_old, y_old, z_old, start_date, sid_interp, PM_index):
    labels = []
    label_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/label/"+str(start_date)+"/"
    PMstrs = ['PM1', 'PM2.5', 'PM10']

    filename = label_dir + sid_interp + "_" + str(start_date) + "_" + PMstrs[PM_index] + "_grid" + str(GRID_SIZE) + ".csv"
    pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=3)
    PM_true = pd_df.values
    
    label_count = (PM_true >= 0).sum() + 1
    
    x_new = np.zeros(label_count)
    y_new = np.zeros(label_count)
    z_new = np.zeros(label_count)
    
    index = 0
    for col in range(GRID_SIZE):
        for row in range(GRID_SIZE):
            PMval = PM_true[row][col]
            if (PMval >= 0):
                index += 1
                x_new[index] = row
                y_new[index] = col
                z_new[index] = PMval
                
    x = np.concatenate((x_old, x_new))
    y = np.concatenate((y_old, y_new))
    z = np.concatenate((z_old, z_new))
    return x, y, z
    
def labelAndPlotPersonalData(map_dir, PM_pred, sid_plot, PM_index, start_date):
    label_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/label/"+str(start_date)+"/"
    labels = []
    filename = label_dir + sid_plot + "_" + str(start_date) + "_" + PMstrs[PM_index] + "_grid" + str(GRID_SIZE) + ".csv"

    if (os.path.isfile(filename)):
        if (MAPPING_ON):
            pred_map = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
            final_map_dir = map_dir + sid_plot
            createMap(pred_map, start_date, sid_plot, PM_pred)
            pred_map.save(map_dir + str(start_date) + "_" + str(sid_plot) + ".html")
            
        pd_df=pd.read_csv(filename, sep=',',header=None, skiprows=3)
        PM_true = pd_df.values
        error = calculateError(PM_true, PM_pred)
        errors.append(error)
        r2s.append(calculateR2(PM_true, PM_pred))

def calculateError(PM_true, PM_pred):
    mask = PM_true > 0
    PM_mask = PM_true[mask]
    PM_diff = np.zeros((GRID_SIZE, GRID_SIZE))
    errors2 = []

    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            if (PM_true[row][col] > 0):
                PM_diff[row][col] = PM_true[row][col] - PM_pred[row][col]
                errors2.append(np.abs(PM_diff[row][col]) / PM_true[row][col])
            else:
                PM_diff[row][col] = 0
                PM_true[row][col] = 0
                
    average_error = np.average(errors2) * 100
    return average_error

def calculateR2(PM_true, PM_pred):
    mean_true = np.average(PM_true)
    
    PM_SStot = np.zeros((GRID_SIZE, GRID_SIZE))
    PM_SSres = np.zeros((GRID_SIZE, GRID_SIZE))
    
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            if (PM_true[row][col] > 0):
                PM_SStot[row][col] = (PM_true[row][col] - mean_true) ** 2
                PM_SSres[row][col] = (PM_true[row][col] - PM_pred[row][col]) ** 2
                
    r2 = 1 - (np.sum(PM_SSres) / np.sum(PM_SStot))
    return r2
                        
def createMap(mapObj, start_date, sid, PM_pred):
    end_date = getEndDate(start_date)
    data_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/raw/personal/"+str(start_date)+"-"+str(end_date)+"/"
    sids = ['XXM007', 'XXM008']
    pdata = data_downloader.readAirSpeckPCSV(start_date, end_date, data_dir)
    belt_index = sids.index(sid)
    
    maxPM = np.max(PM_pred)
    minPM = np.min(PM_pred)
    
    ##Add prediction cells
    for num, cell in enumerate(cells):
        row = int(num / GRID_SIZE)
        col = int(num % GRID_SIZE)
        PM = PM_pred[row][col]
        cell_multi = cell[0]["geometry"]
        osm_reader.addCellPM(mapObj, cell_multi, PM, maxPM, minPM)
            
    ##Add validation walk
    for j in range(len(pdata[belt_index])):
        folium.CircleMarker((pdata[belt_index]["latitude"][j], pdata[belt_index]["longitude"][j]),
                    radius=5,
                    color='#000000',
                    weight=1.0,
                    fill_color=osm_reader.assignColor(pdata[belt_index]["PM2.5"][j], maxPM, minPM),
                    fill=True,
                    fill_opacity=1.0,
                   ).add_to(mapObj)

def getEndDate(start_date):
    return start_date + 1